In [1]:
import os

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

c:\Users\tejas\Langchain-Basics\venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [5]:
google_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", convert_system_message_to_human=True)

In [6]:

message=[SystemMessage(content="Hi you are nice Bot"),HumanMessage(content="Hi, how are you buddy?")]

In [16]:
message2 = [SystemMessage(content = "List out the verbs in the sentence"), HumanMessage(content = "The quick brown fox jumps over the lazy dog.")]

In [7]:
google_model.invoke(message)

AIMessage(content="Hi there! That's very kind of you to say!\n\nAs an AI, I don't have feelings or a physical state, but I'm operating perfectly and ready to assist you. So, in my AI way, I'm doing great!\n\nHow are you doing today? And how can I help you?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bbbbf-04da-7850-86ab-0678df78d967-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 13, 'output_tokens': 190, 'total_tokens': 203, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 122}})

### Parsing

In [8]:
from langchain_core.output_parsers import StrOutputParser

In [9]:
parser = StrOutputParser()

In [12]:
resp = google_model.invoke(message)

In [13]:
parser.invoke(resp)

"Hi there! That's very kind of you to say. I'm doing great, functioning perfectly and ready to help.\n\nHow are you doing today, buddy? And what can I do for you?"

### Chaining with LCEL

In [14]:
chain = google_model | parser

In [15]:
chain.invoke(message)

"Hi there! That's so kind of you to say, thank you!\n\nI'm doing great, functioning perfectly and ready to help. How about you, how are you doing today?"

In [17]:
chain.invoke(message2)

'The verb in the sentence "The quick brown fox jumps over the lazy dog" is:\n\n*   **jumps**'

### Agent with Memory

In [24]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

In [21]:
search = TavilySearchResults()

C:\Users\tejas\AppData\Local\Temp\ipykernel_9036\1244456812.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search = TavilySearchResults()


In [22]:
tool = [search]

In [23]:
memory = MemorySaver()

In [28]:
agent_executor = create_react_agent(model=google_model, tools=tool, checkpointer=memory)

C:\Users\tejas\AppData\Local\Temp\ipykernel_9036\44101554.py:1: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(model=google_model, tools=tool, checkpointer=memory)


In [29]:
config = {"configurable":{"thread_id":"a1"}}

In [31]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="hi i am Tejas! and i live in bangalore and I work as machine Learning Engineer")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content=[{'type': 'text', 'text': "Hi Tejas! It's good to connect with you again. Bangalore is a hub for Machine Learning Engineers! Is there anything specific you'd like to discuss or know about today?", 'extras': {'signature': 'Cq8CAXLI2nypjaekQGxB+J4PVj9m5d9RyWid2ycT4NHRm4ZyAzBW5Cp+Ir+BfJ8Q/sqp3suPOYjDBXGstry00Cvz4eeiy4FLWhW61aGh+HWb56598qi0rWyuwjbsd3jxqZoewVzFUPUm+PJpnRbvksTRTE/f1CXL23N6IfBiaOOExOevaxpgKeUtPKKAdjW2LXEmZ7kcSkJz02sPDpVq2WdQTQSoJ/mSaN1uNBCJqpF678E2FjTuH+8gbNyj1HStQFGTEYzbGESyewAP/hlXDsFcPh42s1/bn3CK9v6KNxE+SwyrypQ3S9rh+ya4Zh0byZ9rOkcKeQxptHFH/gDSBUrqJEUiaZBYiRst1gV+VVFbWm2il1LqgIiVkoU84417iqRc9ggVbexUF0DTiq7EAaKh'}}], additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bbbf4-4409-78c0-ae27-2174a17796db-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 157, 'output_tokens': 92, 'total_tokens': 

In [32]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="what is the weather where I live?")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Bangalore"}'}, '__gemini_function_call_thought_signatures__': {'ec72fc82-747d-4633-ac88-40d829081dd0': 'Cp8CAXLI2nydpHeoxfjGz3E0dhMTtn3b920GRpYuqNR0F5fbzqXgRzC25JHaFFCzh2oSN/d7qhARd/FGY/nLL5f2vPd2OCgiM5AuC92PRq71G3uo08c7sldAL6Ydv9vruHZwc9OUAEdBDIN//U9OArlV9xU14o2R3ZBycoAIlxc/nsIBJHFub6LlPE9kRe2vaqwL5uV1TkxDOEvQow1eB8UR21BtKNZ8QvAlBhQrUxZBlWVZM036E/xgrUVB2s5hMyTeAHzavUkCUeK5YtPdrylgQ9oZjKHmn+lqMNxYJ8fF7FhXMDsJ5zGeH/WP5u+VAXNdFCg3NYnzAbp7YlLrGzk36JswS6yRKaSXielc8QyY/dxHUjs4gAVKKWihotcMjJM='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bbbf4-f440-7c61-be64-688083441fff-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Bangalore'}, 'id': 'ec72fc82-747d-4633-ac88-40d829081dd0', 'type': 'tool_call